# Скачивание и разметка собственного корпуса

Этот набор упражнений должен помочь вам научиться:

- выгружать интересующие вас тексты из Интернета
- сохранять их в формате, удобном для обработки
- выполнять их разметку

[Ссылка](https://colab.research.google.com/drive/1N6pky6LG1pkJ9_VVAFUB1fpxWRq5KfrY?usp=sharing) на тетрадку в Google Colab

In [2]:
#Скачиваем пакеты, которых нет в стандартной библиотеке Python
!pip install fake-useragent
!pip install tqdm
!pip install newscatcher

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 KB 1.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.6/138.6 KB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.0/49.0 KB 6.8 MB/s eta 0:00:00
  Created wheel for feedparser: filename=feedparser-5.2.1-py3-none-any.whl size=44951 sha256=ee16d6c86e5b695efcf730ffda195112b324719e0b7d05a3bf579298385d567a
  Stored in directory: /root/.cache/pip/wheels/67/38/07/59c89c3d334e7f1f743898af6d21c15ecb3588ad04af7ddee0
Successfully built feedparser


Импортируем нужные для работы модули

In [3]:
#Собственно выкачивание файлов из Интернета
import requests

#Работа с таблицами — понадобится, чтобы хранить наш корпус
import pandas as pd

#Выполнение быстрых математических вычислений — пригодится, потому что иначе
#корпус с большим количеством текстов может долго обрабатываться
import numpy as np

# Модуль для работы с HTML-файлами
from bs4 import BeautifulSoup

# Модуль для скачивания новостей
import newscatcher

# Модуль для работы с регулярными выражениями
import re

#Функция, который измеряет время исполнения фрагмента программы — без неё
#в принципе тоже всё будет работать
from tqdm.auto import tqdm

#Поможет замаскироваться под обычного пользователя в Интернете
from fake_useragent import UserAgent 

#Переменные, нужные, чтобы маскироваться под обычного пользователя Интернета
ua = UserAgent(verify_ssl=False)
headers = {'User-Agent': ua.random}

## Сбор текстов в Интернете

Python позволяет достаточно простым образом скачивать страницы из интернета по ссылке. Например, ниже скачивается и записывается в переменную файл с предыдущего занятия.

In [4]:
akarenina_link = 'https://raw.githubusercontent.com/alekseyst/text_analysis_2023/main/Seminar_3/karenina.txt'
akarenina = requests.get(akarenina_link, headers=headers)
akarenina

<Response [200]>

In [5]:
#Как мы видим, результат запроса представляет собой какой-то объект.
#Чтобы извлечь его содержимое, нужно обратиться к атрибуту text.

akarenina.text[:400]

'\r\n Л.Н. Толстой. Анна Каренина \r\n\r\n Роман в восьми частях \r\n\r\n\t Мне отмщение, и Аз воздам  \r\n\t \r\n\r\n Часть первая \r\n\r\n I \r\n\r\n Все счастливые семьи похожи друг на друга, каждая несчастливая семья несчастлива по-своему. \r\n Все смешалось в доме Облонских. Жена узнала, что муж был в связи с бывшею в их доме француженкою-гувернанткой, и объявила мужу, что не может жить с ним в одном доме. Положение это '

##HTML

Если мы, однако, имеем дело не с txt-файлами, выложенными в Интернет, а с обычными страницами, приходится иметь дело с HTML.

In [8]:
#HTML-страница новости

news_cher_link = "https://cher-is.com/izvestnaya-nejroset-sostavila-obrazy-cherepovtsa-i-vologdy/"
html_cher_request_result = requests.get(news_cher_link, headers=headers)
html_cher = html_cher_request_result.text
html_cher[:800]

'<!DOCTYPE html>\n<html lang="ru-RU" itemscope itemtype="http://schema.org/WebSite" prefix="og: http://ogp.me/ns#">\n<head> <a href="https://metrika.yandex.ru/stat/?id=52125751&amp;from=informer" target="_blank" rel="nofollow"><img src="https://informer.yandex.ru/informer/52125751/3_1_FFFFFFFF_EFEFEFFF_0_pageviews" style="width:88px; height:31px; border:0;" alt="Яндекс.Метрика" title="Яндекс.Метрика: данные за сегодня (просмотры, визиты и уникальные посетители)" class="ym-advanced-informer" data-cid="52125751" data-lang="ru" /></a>   <script type="text/javascript"> (function(m,e,t,r,i,k,a){m[i]=m[i]||function(){(m[i].a=m[i].a||[]).push(arguments)}; m[i].l=1*new Date();k=e.createElement(t),a=e.getElementsByTagName(t)[0],k.async=1,k.src=r,a.parentNode.insertBefore(k,a)}) (window, document, "scr'

Конечно, страшно, но не очень.

Если хочется прямо разобраться с HTML, вот, например, [ссылка](https://www.w3schools.com/html/html_intro.asp). А так нам важно знать, что всякие элементы располагаются внутри тегов, и выглядит это как-то так (это тэг \<p\>, который используется для разбивки страницы на абзацы — paragraph):

\<p\>This is a paragraph.\</p\>

Обычно достаточно уметь найти, в каком тэге находится нужная информация, и извлечь её. Для поиска нужно всего лишь нажать на то, что вы хотите извлечь со страницы правой клавишей — Inspect, и после этого вы попадёте в удивительный мир HTML. Но! Вас переведут ровно к тому элементу, который вы ткнули. После этого, поводив по экрану, на котором подсвечиваются блоки, соответствующие тэгам, можно наиболее точно найти то, что вас интересует.

Новость выкачанной нами страницы, включая заголовок и картинку, находится в тэге \<div class="blog-lg-box"\>Содержимое\</div\>. Обратите внимание, в файле очень много тэгов 

Для извлечения можно использовать библиотеку BeautifulSoup, которую мы уже импортировали. 

In [ ]:
#Просим модуль проанализировать наш файл
html_cher_soup = BeautifulSoup(html_cher, 'html.parser')

#Ищем
body_cher_soup = html_cher_soup.find("div", class_="media-body")

#Тэг — первая переменная, класс — вторая
#Если хотите найти несколько тэгов с одинаковыми классами, используйте find_all
#Они будут выданы в виде списка объектов BeautifulSoup — из каждого придётся
#извлекать текст

body_cher_soup

<div class="media-body">
<div class="blog-lg-box">
<a class="img-responsive" href="https://cher-is.com/izvestnaya-nejroset-sostavila-obrazy-cherepovtsa-i-vologdy/" title="Известная нейросеть составила образы Череповца и Вологды">
<img alt="" class="img-responsive wp-post-image" height="408" loading="lazy" sizes="(max-width: 264px) 100vw, 264px" src="https://cher-is.com/wp-content/uploads/2023/02/cherepovets.png" srcset="https://cher-is.com/wp-content/uploads/2023/02/cherepovets.png 264w, https://cher-is.com/wp-content/uploads/2023/02/cherepovets-194x300.png 194w" width="264"/>
</a></div>
<div class="blog-post-lg">
<a href="https://cher-is.com/author/admin/"><img alt="" class="img-responsive img-circle avatar-40 photo" height="40" loading="lazy" src="https://secure.gravatar.com/avatar/12d14aad72f86a5c2271b4922c91b55f?s=40&amp;d=mm&amp;r=g" srcset="https://secure.gravatar.com/avatar/12d14aad72f86a5c2271b4922c91b55f?s=80&amp;d=mm&amp;r=g 2x" width="40"/></a>
Автор:<a href="https://cher-is

Уже похоже на новость! Если теперь удалить все оставшиеся тэги, которые нас явно не интересуют, можно получить собственно текст.

In [ ]:
cher_news = re.sub('<[^<]+?>', '', body_cher_soup.text).strip()

#Как это работает? Регулярное выражение заменяет все (практически все) тэги на
#пустую строку. Метод .strip() удаляет лишние переносы строк и подобное по краям
#новости
#Осторожно! Если вы занимаетесь чем-то серьёзным и работаете с потенциально 
#опасными сайтами, удалять тэги следует другим способом.

cher_news

'Автор:ЧИ\nMidjourney, нейросеть, Череповец\n\nИзвестная нейросеть составила образы Череповца и Вологды\nНейросеть Midjourney изобразила города России в образах, и получились герои для мрачных сказок.\nНа этот раз автор телеграм-канала «Нейросеть видит» сформировал запрос в Midjourney так, чтобы нейросеть показала российские города в человеческих обличьях. Например, две столицы, Москва и Санкт-Петербург, выглядят как модная пара. Девушка — в стильном кокошнике, а парень с уложенными бородой и усами.\nЧереповец изображён в виде монаха, а Вологда — знатной женщины.'

Чтобы избавиться от автора, названия картинки и остального, можно их удалить в полученном выше тексте или извлекать из текста новости только то, что содержится в тэгах \<p\>\</p\>.

In [ ]:
cher_news_no_author = ''

#Приходится использовать цикл, потому что метод find_all выдаёт как результат
#не строки, а объекты модуля Beautiful Soup

for p in body_cher_soup.find_all("p"): 
  cher_news_no_author += p.text.strip() 

cher_news_no_author

'Нейросеть Midjourney изобразила города России в образах, и получились герои для мрачных сказок.На этот раз автор телеграм-канала «Нейросеть видит» сформировал запрос в Midjourney так, чтобы нейросеть показала российские города в человеческих обличьях. Например, две столицы, Москва и Санкт-Петербург, выглядят как модная пара. Девушка — в стильном кокошнике, а парень с уложенными бородой и усами.Череповец изображён в виде монаха, а Вологда — знатной женщины.'

## Задание

Выкачайте одну любую новость с сайта ИА Панорама. Найдите, где в HTML-файле находится текст и сохраните его в переменную.

In [ ]:
#Впишите сюда код



## Одна новость — хорошо, но мало

Чтобы сделать корпус, выкачивания одной интернет-страницы, конечно, недостаточно. Хотелось бы выкачивать сразу много текстов — или в целом из Интернета, или хотя бы с одного сайта.

В целом, выкачать всю нужную информацию с сайта можно, найдя закономерность в том, как называются её страницы. Иногда это просто (например, каждая из новостей может быть просто пронумерована), иногда требуется больше изощрённости.

In [ ]:
from datetime import datetime, timedelta
    
d = datetime.today() - timedelta(days=1)

In [ ]:
date_list = [datetime.today() - timedelta(days=x) for x in range(100)]

In [ ]:
date_list = [f"{day.day}-{day.month}-{day.year}" for day in date_list]

In [ ]:
date_list[:10]

['4-3-2023',
 '3-3-2023',
 '2-3-2023',
 '1-3-2023',
 '28-2-2023',
 '27-2-2023',
 '26-2-2023',
 '25-2-2023',
 '24-2-2023',
 '23-2-2023']

In [ ]:
td = datetime.today()
f"{td.day}-{td.month}-{td.year}"

'4-3-2023'

In [ ]:
dates = []
n = 100
date_list = [datetime.today() - timedelta(days=x) for x in range(n)]
date_list = [f"{date.day}-{date.month}-{date.year}" for date in date_list]

In [ ]:
date_list[-1]

'25-11-2022'

In [ ]:
all_titles = []
all_dates = []
all_links = []
all_texts = []
all_spheres = []

for date in tqdm(date_list):
  page = f"https://panorama.pub/news/{date}"
  result = requests.get(page)
  html = result.text
  soup = BeautifulSoup(html)
  titles = soup.find_all('div', {'class': "pt-2 text-xl lg:text-lg xl:text-base text-center font-semibold"})
  titles = [x.text.strip() for x in titles]
  all_titles.extend(titles)
  all_dates.extend([date]*len(titles))
  links = soup.find_all('a', {'class': "flex flex-col rounded-md hover:text-secondary hover:bg-accent/[.1] mb-2"}, href=True)
  links = [f"https://panorama.pub{link['href']}" for link in links]
  all_links.extend(links)
  for link in links:
    result = requests.get(link)
    html = result.text
    soup = BeautifulSoup(html)
    page_text = soup.find('div', {'itemprop': "articleBody"})
    lines = [line.text.strip() for line in page_text.find_all('p')]
    all_texts.extend([' '.join(lines).replace('\xa0', "").replace('\n', ' ')])
    sphere = soup.find('a', {'class': "badge"})
    if sphere == None:
      all_spheres.append(sphere)
    else:
      all_spheres.append(sphere.text)



  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
all_texts[0]


'Государственная дума приняла в третьем чтении законопроект«О внесении изменении в порядок добычи полезных ископаемых». Начиная с 1 марта этого года, гражданам разрешат добывать на приусадебных участках нефть и природный газ для личного пользования, а также в целях переработки. Согласно выводам экспертов Минэнерго, залежи полезных ископаемых на таких территориях весьма ограничены, поэтому извлечённые объёмы не повлияют как на статистику, так и на цены на рынке нефтепродуктов. Владельцам приусадебных участков также разрешат сдавать земли в аренду нефтегазовым компаниям. Для начала работ по бурению скважин следует оформить соответствующие документы и зарегистрироваться в ФНС в качестве индивидуального предпринимателя или самозанятого. Помимо единого налога, дачникам надлежит заплатить в бюджет НДПИ (налог на добычу природных ископаемых). В случае самовольной добычи природных ископаемых без оформления разрешений или регистрации в налоговом органе продолжат действовать нормы УК РФ.'

In [ ]:
len(all_titles) == len(all_dates)

True

## Слишком сложно?..

К счастью, нередко не нужно руками с нуля писать подобные программы для выкачки сайтов.

Некоторые сайты, в том числе крупные соцсети, сами умеют представлять данные в формате, удобном не для пользователя-человека, а для компьютера. Это анзывается API — см., например, [VK API](https://dev.vk.com/api/overview).

Кроме этого, есть большое количество готовых модулей или инструментов на основе Python. Универсальная, но достаточно сложная в использовании программа, которая скачивает сайты, самостоятельно переходя по ссылкам, которые находит на их странице (это называется Crawler), — [Scrapy](https://scrapy.org/).

Кроме этого, есть модули, которые "из коробки" умеют обкачивать определённые типы сайтов — в том числе новостные. Проще всего пользоваться модулем newscatcher — он не поддерживает Панораму, но умеет работать с большим количеством других сайтов.

In [ ]:
#Небольшая часть из множества русскоязычных сайтов, с которыми работает newscatcher

newscatcher.urls(language = 'ru')[:15]

['yandex.ru',
 'ria.ru',
 'rbc.ru',
 'kommersant.ru',
 'kremlin.ru',
 'kp.ru',
 'rg.ru',
 'lenta.ru',
 'vedomosti.ru',
 'aif.ru',
 'mk.ru',
 'government.ru',
 'regnum.ru',
 'meduza.io',
 'newsru.com']

In [ ]:
#Выкачиваем статьи — получаем особый объект модуля
lenta = newscatcher.Newscatcher('lenta.ru')

#Извлекаем из него собственно статьи
lenta_articles = lenta.get_news()['articles']

len(lenta_articles)

200

In [ ]:
lenta_articles[0]

{'id': 'https://lenta.ru/news/2023/03/04/h3/',
 'guidislink': True,
 'link': 'https://lenta.ru/news/2023/03/04/h3/',
 'authors': [{'name': 'Иван Потапов'}],
 'author': 'Иван Потапов',
 'author_detail': {'name': 'Иван Потапов'},
 'title': 'Япония перенесла первый пуск H3 из-за погоды',
 'title_detail': {'type': 'text/plain',
  'language': None,
  'base': 'https://lenta.ru/rss/',
  'value': 'Япония перенесла первый пуск H3 из-за погоды'},
 'links': [{'rel': 'alternate',
   'type': 'text/html',
   'href': 'https://lenta.ru/news/2023/03/04/h3/'},
  {'type': 'image/jpeg',
   'length': '31892',
   'href': 'https://icdn.lenta.ru/images/2023/03/04/10/20230304100550304/pic_89e8a7ac4c302716a884364c27b7be59.jpeg',
   'rel': 'enclosure'}],
 'summary': 'Первый пуск новой ракеты H3 перенесен с 6 на 7 марта из-за погодных условий в районе космодрома Танэгасима юго-западной префектуры Кагосима. Об этом сообщает Японское агентство аэрокосмических исследований. Старт планируется осуществить в промежутке

In [ ]:
print(lenta_articles[0]['title'])
print(lenta_articles[0]['link'])

Япония перенесла первый пуск H3 из-за погоды
https://lenta.ru/news/2023/03/04/h3/


## Задание

Выкачайте с помощью newscatcher несколько новостей с любого сайта и извлеките из них текст в отдельную переменную.

In [ ]:
# Впишите сюда код

## Экспорт текстов

Осталось сохранить полученные тексты, чтобы не выкачивать каждый раз и иметь доступ в любой момент. Для этого оформим их в виде таблицы, которая содержит всю метаминформацию.

In [ ]:
#Создаём индекс, чтобы пронумеровать каждый текст (это нам понадобится позже)

index = [a for a in range(len(all_dates))]

#Добавляем в особую табличку тексты. Для этого подаём функции DataFrame словарь
#В котором на первом месте — название столбца, на втором — содержимое столбца
#Все столбцы должны быть одинаковой длины
corpus = pd.DataFrame({'index': index, 'date': all_dates, 'sphere': all_spheres,
                       'title': all_titles, 'text': all_texts,  
                       'link': all_links})

corpus.head()

,index,date,sphere,title,text,link
0,0,4-3-2023,Общество,Гражданам разрешили добывать нефть и газ на св...,Государственная дума приняла в третьем чтении ...,https://panorama.pub/news/grazdanam-razresili-...
1,1,4-3-2023,Политика,Китай подал заявку на вступление в Союзное гос...,Заведующий Канцелярией Комиссии ЦК КПК по инос...,https://panorama.pub/news/kitaj-podal-zaavku-n...
2,2,4-3-2023,Политика,Деколонизаторки из Франции потребовали вернуть...,Активисты движения «Смерть колониализму» из Па...,https://panorama.pub/news/dekolonizatorki-iz-f...
3,3,4-3-2023,Политика,Победа России: Госдума пересмотрела результаты...,Государственная дума приняла постановление «Об...,https://panorama.pub/news/gosduma-peresmotrela...
4,4,3-3-2023,Общество,Бездетных россиян будут ежегодно штрафовать,Государственная дума приняла в первом чтении з...,https://panorama.pub/news/bezdetnyh-rossian-pr...


In [ ]:
corpus.to_csv('panorama_corpus.tsv', sep = '\t', encoding='utf-8',  index = False)